# Aeropressure Mapping
---
![](img/banner.png)

In [1]:
# N A S T R A N- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
import pyNastran
from pyNastran.op2.op2 import OP2
from pyNastran.bdf.bdf import BDF, read_bdf
from pyNastran.utils import object_attributes, object_methods

In [2]:
# Classical - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
import pandas as pd
import numpy as np
import os
import datetime
import glob
import re
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import scipy
from scipy.interpolate import griddata as gd

In [3]:
print('='*60)
print('%-16s%-16s%-16s%-16s' % ('pandas','pyNastran','numpy', 'scipy'))
print('-'*12,'  ','-'*12,'  ','-'*12,'  ','-'*12)
print('%-16s%-16s%-16s%-16s' % (pd.__version__,pyNastran.__version__,np.__version__,scipy.__version__))
print('='*60)

pandas          pyNastran       numpy           scipy           
------------    ------------    ------------    ------------
0.25.1          1.2.0           1.16.5          1.3.1           


## 1. $\mathrm{NASTRAN}$ Structural Mesh

In [4]:
# User's barecode
to='to77222'
# Working directory
wd='C:\\Users\\'+str(to)+'\\Desktop\\\Basic-FEA-Toolkits\\NASTRAN-Cp-field-Interpolation\\A-Jupyter-Notebook\\'

In [5]:
# Minimal Working Example
# Creation of a BDF object
bdf=BDF()

In [6]:
bdf.read_bdf(wd+'mwe\\'+'plate.dat', xref=True)

In [7]:
# Example - Loop over the GRID and Print the Coordinates...
for i,n in enumerate(bdf.nodes.keys()):
    print(bdf.nodes[n].xyz)

[0. 0. 0.]
[1. 0. 0.]
[1. 1. 0.]
[0. 1. 0.]


In [8]:
# Example - Compute the Centre of Gravity of the elements...
CG={}
for i,e in enumerate(bdf.elements.keys()):
    CG[e]=(bdf.nodes[bdf.elements[e].nodes[0]].xyz+bdf.nodes[bdf.elements[e].nodes[2]].xyz)/2.

In [9]:
CG

{1: array([0.5, 0.5, 0. ])}

## 2. Aeropressure $C_p$ field 

In [10]:
# Example - Upload pressure into a dataframe
aero=pd.read_csv(wd+'mwe\\'+'9001.csv', names=['Topo','x','y','z','Cp'], header=None)

In [11]:
aero

,Topo,x,y,z,Cp
0,4,0.5,0.5,0.0,0.707107
1,4,1.5,0.5,0.0,0.707107
2,4,1.5,1.5,0.0,0.707107
3,4,0.5,1.5,0.0,0.707107


## 3. Interpolation of Aeropressure $C_p$ field

In [12]:
# Example - Assess pressure @CG
PLOAD={}
for k,c in enumerate(CG):
    PLOAD[c]=gd(aero[['x', 'y', 'z']].values, aero['Cp'].values, CG[c], method='nearest')

## 4. $\mathrm{NASTRAN}$ $\mathrm{PLOAD4}$ Output

In [13]:
pid='9001'

In [14]:
# Example - Create PLOAD4 Cards
for k,c in enumerate(CG):
    bdf.add_pload4(pid,c,PLOAD[c]*np.ones(4))

In [15]:
bdf.write_bdf(wd+'mwe\\'+'plate_with_pload4.dat', interspersed=True, size=8, is_double=False)

## The end.